# Oscilloscopio

Il notebook mostra come sia possible riprodurre un oscilloscopio interattivo per visualizzare in continua delle acquisizioni dell'\texttt{ADC}. Lo script apre una finestra interattiva in cui è possibile interagire con l'oscilloscopio nel seguente modo:

* Mouse scroll: permtte di controllare lo zoom biassiale quando il mouse si trova nel grafico, uniassiale quando si trova su uno degli assi.
* Tasto `'a'` forza un *autoscale* del grafico
* Tasto `'.'` visualizza le tracce dell'oscilloscopio come punti
* Tasto `'-'` visualizza le tracce dell'oscilloscopio come linee
* Tasto `'x'` esporta l'ultima misura su file (nome del file da indicare su prompt della shell python)
* Tasto `'space'` entra/esce da pausa nell'acquisizione
* Tasto `'enter'` fa una singola acquisizione
* Tasto `'esc'` termina il programma

I parametri di acquisizione e le forme d'onda del generatore vanno impostate nel codice con le semplici istruzioni indicate sotto.

In [43]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
## Modifiche per muovere il programma

import serial
import time

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2(iconfig = 3)
# ad2.vdd = +5
# ad2.power(True)
#   2. Configurazione generatore di funzioni
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.config(ampl=5.0, freq=4e4, phi=0.0, func=tdwf.funcSine, duty=50, offs=0)
# wavegen.w2.config(ampl=2.5, freq=1e2, phi=0.0, func=tdwf.funcDC, duty=50, offs=3)
# wavegen.w2.sync()
wavegen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 0.5e6
scope.npt = 1000000
scope.ch1.rng = 50
scope.ch2.rng = 50
scope.ch1.avg = True
scope.ch2.avg = True
scope.trig(True, level=0.0, sour=tdwf.trigsrcCh2, delay=0, hist=0.1)

# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_scroll(event):
    kk = 1.5  # fattore di zoom/dezoom
    # [1] Calcolo delle coordinate del mouse rispetto agli assi
    x0, x1 = ax.get_xlim()  # limiti asse x
    y0, y1 = ax.get_ylim()  # limiti asse y
    figw, figh = fig.get_size_inches() * fig.dpi  # calcola dimensioni finestra
    box = ax.get_position()  # posizione assi nella finestra
    xdata = (event.x/figw-box.x0)/(box.x1-box.x0)*(x1-x0)+x0
    ydata = (event.y/figh-box.y0)/(box.y1-box.y0)*(y1-y0)+y0
    # [2] del fattore di zoom
    if event.button == 'up':
        factor = 1 / kk
    elif event.button == 'down':
        factor = kk
    else:
        return
    # [3] calcolo delle nuove coordinate limite degli assi
    newdx = (x1-x0) * factor  # nuovo span asse x
    relx = (x1 - xdata) / (x1-x0)  # posizione relativa mouse nello span
    newdy = (y1-y0) * factor  # nuovo span assey
    rely = (y1 - ydata) / (y1-y0)  # posizinoe relativa mouse nello span
    if xdata > x0:  # zoom x
        ax.set_xlim([xdata - newdx * (1-relx), xdata + newdx * (relx)])
    if ydata > y0:   # zoom y
        ax.set_ylim([ydata - newdy * (1-rely), ydata + newdy * (rely)])
    # [4] aggiorna la figura
    fig.canvas.draw()
    fig.canvas.flush_events()

def on_key(event):
    global flag_run, flag_acq, hp1, hp2
    if event.key == 'a':  # autoscale
        ax.set_xlim([tt.min(), tt.max()])
        ax.set_ylim([min(min1.min(), min2.min()), max(max1.max(), max2.max())])
    if event.key == 'x':  # export su file
        filename = input("Esporta dati su file: ")
        data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))
        info = f"Acquisizione Analog Discovery 2\nTimestamp {scope.time.dt}\ntime\tch1\tch2"
        np.savetxt(filename, data, delimiter='\t', header=info)
    if event.key == '.':  # plot a punti
        hp1.set_linestyle("")
        hp1.set_marker(".")
        hp2.set_linestyle("")
        hp2.set_marker(".")
    if event.key == '-':  # plot a linea
        hp1.set_linestyle("-")
        hp1.set_marker("")
        hp2.set_linestyle("-")
        hp2.set_marker("")
    if event.key == ' ':  # run/pausa
        flag_acq = not flag_acq
    if event.key == 'enter':  # acqusizione singola
        flag_acq = False
        scope.sample()
    if event.key == 'escape':  # termina programma
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, ax = plt.subplots(figsize=(12, 6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('scroll_event', on_scroll)
fig.canvas.mpl_connect('key_press_event', on_key)
#   2. Creazione dei vettori dei tempi
# (troppi punti, media uno ogni 4)
tt = 1e3*scope.time.vals.reshape(-1, 4).mean(axis=1)
# (vettore degli errori: buffer più corto di un fattore 4)
tte = np.repeat(tt[::4], 2)
tte = np.append(tte[1:]-8e3*scope.time.dt, tte[-1]+8e3*scope.time.dt)
#   3. Ciclo di misura

tt_vals = [] # salvataggio lista tempi
chout_vals = [] # salvataggio lista valori ch2
tt_vals_reshape = [] # salvataggio lista tempi
chout_vals_reshape = [] # salvataggio lista valori ch2

flag_first = True
flag_acq = True
flag_run = True

# Pilotaggio carrello
ser = serial.Serial()
ser.port = "COM5"
ser.baudrate = 115200   
ser.timeout = 0
ser.rts = False
ser.dtr = False
ser.open()
#ser = serial.Serial('COM11', 115200, timeout=1, dsrdtr = False)
#time.sleep(2)  # Attendi che la connessione si stabilisca (utile per dispositivi come Arduino)
for i in range(11):
    response = ser.readline().decode('utf-8')
    print(response[:-1])

# Manda il comando
# ser.write(b'X 0\n') # setta lo zero alla posizione attuale
# ser.write(b'X?\n') # richiesta posizione
ser.write(b'TX 0\n') # TX = target ([0, 80000]), DX = spostamento
ser.write(b'SX 5000.0\n') # setta la velocità. Velocità di default = 4000 microstep al secondo
# ser.write(b'SX?\n') # non funziona

# Attendi e leggi la risposta
response = ser.readline().decode('utf-8').strip()
print(f"Risposta ricevuta: {response}")

while flag_run:
    if flag_acq:  # SE la misura è attiva (space)
        time.sleep(2)
        scope.sample()
    # Calcolo dei vettori da visualizzare (vengono fatte alcune medie)
    min1 = np.repeat(scope.ch1.min.reshape(-1, 2).mean(axis=1), 2)
    max1 = np.repeat(scope.ch1.max.reshape(-1, 2).mean(axis=1), 2)
    min2 = np.repeat(scope.ch2.min.reshape(-1, 2).mean(axis=1), 2)
    max2 = np.repeat(scope.ch2.max.reshape(-1, 2).mean(axis=1), 2)
    ch1 = scope.ch1.vals.reshape(-1, 4).mean(axis=1)
    ch2 = scope.ch2.vals.reshape(-1, 4).mean(axis=1)
    tt_vals.append(scope.time.vals)
    chout_vals.append(scope.ch1.vals)
    tt_vals_reshape.append(tt)
    chout_vals_reshape.append(ch1)
    
    if flag_first:
        # Prima esecuzione: creazione grafici # commentato il canale 2, che non serve
        flag_first = False
        # hp1, = plt.plot(tt, ch1, "-", label="Ch1", color="tab:orange", linewidth = 1.2)
        # hp2, = plt.plot(tt, ch2, "-", label="Ch2", color="tab:blue", linewidth = 0.8)
        # hp3 = plt.fill_between(tte, min1, max1, color='tab:orange', alpha=0.3)
        # hp4 = plt.fill_between(tte, min2, max2, color='tab:blue', alpha=0.3)
        # path1 = hp3.get_paths()[0]
        # path2 = hp4.get_paths()[0]
        # plt.legend()
        # plt.grid(True)
        # plt.xlabel("Time [msec]", fontsize=15)
        # plt.ylabel("Signal [V]", fontsize=15)
        # plt.title("User interaction: a|-|.|x|space|enter|escape|scroll")
        # plt.show(block=False)
        # plt.tight_layout()
        # ax.set_xlim([tt.min(), tt.max()])
        # ax.set_ylim([min(min1.min(), min2.min()), max(max1.max(), max2.max())])
    else:
        # Esecuzioni successive: aggiornamento grafici
        # hp1.set_ydata(ch1)
        # hp2.set_ydata(ch2)
        # tmp1 = np.concatenate(([max1[0]], min1, [max1[-1]], max1[::-1], [max1[0]]))
        # path1.vertices[:, 1] = tmp1
        # tmp2 = np.concatenate(([max2[0]], min2, [max2[-1]], max2[::-1], [max2[0]]))
        # path2.vertices[:, 1] = tmp2
        # fig.canvas.draw()
        # fig.canvas.flush_events()
        None

#   4. Chiude figura e porta seriale e libera AD2
plt.close(fig)
# ser.close()
ad2.close()

Dispositivo #1 [SN:210321B1999E, hdwf=1] connesso!
Configurazione #3











Risposta ricevuta: 


KeyboardInterrupt: 

In [44]:
len(chout_vals)


4

In [45]:
# Salvataggio dati per lista di liste in cui ogni lista interna è un'acquisizione

# data = tt_vals + chout_vals # unisce le liste di tempi e ch2 in un'unica lista (forse tt_vals sono tutte uguali, verificare)
data = [tt_vals[0]] + [chout_vals[0]]

with open(r"C:\Users\loreg\Desktop\dati_doppler_no_reshape_avanti_2.txt", "w") as f: # usando "w" riscriviamo da zero, usando "a" aggiungiamo righe
    for row in range(len(data[0])): # le righe sono le posizioni degli elementi in ogni lista interna alla lista di liste
        for column in range(len(data)): # le colonne solo la posizione delle liste nella lista di liste
            f.write(f"{data[column][row]:.9f}\t") # seleziona prima la colonna e poi la riga # inserire [row]:.9f per tagliare i dati alla nona cifra decimale
        f.write(f"\n")



In [ ]:
data_tt = np.column_stack(tt_vals)
data_chout = np.column_stack(chout_vals)
np.savetxt("dati_doppler_1_tt.txt", data_tt, delimiter = "\n", header = "tt vals")
np.savetxt("dati_doppler_1_chout.txt", data_chout, delimiter = "\n", header = "chout vals")